In [ ]:
from datetime import datetime
from getpass import getpass
from cryptography.fernet import Fernet
import pandas as pd
import random

from myproject import app,db
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask import render_template, redirect, request, url_for, flash,abort
from flask_login import login_user,login_required,logout_user
from myproject.models import User,products
from myproject.forms import LoginForm, RegistrationForm,AddForm,delform,updateform,searchform,sellform
from werkzeug.security import generate_password_hash, check_password_hash
from sqlalchemy.orm import sessionmaker,relationship



@app.route('/')
def home():
    return render_template('home.html')


@app.route('/welcome')
@login_required
def welcome_user():
    return render_template('welcome_user.html')


@app.route('/search', methods=['GET', 'POST'])
@login_required
def search():
    form = searchform()
    #Barcode = products(barcode=form.barcode.data)
    Barcode=form.barcode.data
    Name=form.name.data
    Category=form.category.data
    if Barcode!='':
        data=db.engine.execute("select name from product where barcode=?",Barcode)
        names = [row[0] for row in data]
        data1=db.engine.execute("select category from product where barcode=?",Barcode)
        names1 = [row[0] for row in data1]
        data2=db.engine.execute("select barcode from product where barcode=?",Barcode)
        names2 = [row[0] for row in data2]
        data3=db.engine.execute("select price from product where barcode=?",Barcode)
        names3 = [row[0] for row in data3]
        data4=db.engine.execute("select quantity from product where barcode=?",Barcode)
        names4 = [row[0] for row in data4]
    elif Name!='':
        data=db.engine.execute("select name from product where name=?",Name)
        names = [row[0] for row in data]
        data1=db.engine.execute("select category from product where name=?",Name)
        names1 = [row[0] for row in data1]
        data2=db.engine.execute("select barcode from product where name=?",Name)
        names2 = [row[0] for row in data2]
        data3=db.engine.execute("select price from product where name=?",Name)
        names3 = [row[0] for row in data3]
        data4=db.engine.execute("select quantity from product where name=?",Name)
        names4 = [row[0] for row in data4]
    elif Category!='':
        data=db.engine.execute("select name from product where category=?",Category)
        names = [row[0] for row in data]
        data1=db.engine.execute("select category from product where category=?",Category)
        names1 = [row[0] for row in data1]
        data2=db.engine.execute("select barcode from product where category=?",Category)
        names2 = [row[0] for row in data2]
        data3=db.engine.execute("select price from product where category=?",Category)
        names3 = [row[0] for row in data3]
        data4=db.engine.execute("select quantity from product where category=?",Category)
        names4 = [row[0] for row in data4]
    else:
        return redirect(url_for('welcome_user'))
    #sql = "delete from product where barcode='%s'"
    #data=db.engine.execute(sql,Barcode)
    flash('Search the product ')
    return render_template('search.html',names=names,names1=names1,names3=names3,names2=names2,names4=names4,form=form)


@app.route('/add_product', methods=['GET', 'POST'])
@login_required
def add_product():
        form = AddForm()

        if form.validate_on_submit():
            Product = products(name=form.name.data,
                        category=form.category.data,
                        description=form.description.data,
                        barcode=form.barcode.data,
                        price=form.price.data,
                        quantity=form.quantity.data)
            Name=form.name.data
            Category=form.category.data
            Description=form.description.data
            Barcode=form.barcode.data
            Price=form.price.data
            Quantity=form.quantity.data
            data=db.engine.execute("insert into product(name,category,description,barcode,price,quantity) values(?,?,?,?,?,?)",Name,Category,Description,Barcode,Price,Quantity)
            #db.session.add(Product)
            db.session.commit()
            flash('Thanks for adding the new product')
            return redirect(url_for('welcome_user'))
        return render_template('add_product.html', form=form)
    
    
@app.route('/logout')
@login_required
def logout():
    logout_user()
    flash('You logged out!')
    return redirect(url_for('home'))


@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # Grab the user from our User Models table
        user = User.query.filter_by(email=form.email.data).first()


        if user.check_password(form.password.data) and user is not None:
            #Log in the user

            login_user(user)
            flash('Logged in successfully bro.')
            next = request.args.get('next')

            # So let's now check if that next exists or no, otherwise it'll go to
            # the welcome page.
            if next == None or not next[0]=='/':
                next = url_for('welcome_user')

            return redirect(next)
    return render_template('login.html', form=form)


@app.route('/register', methods=['GET', 'POST'])
def register():
    form = RegistrationForm()

    if form.validate_on_submit():
        user = User(email=form.email.data,
                    first_name=form.first_name.data,
                    last_name=form.last_name.data,
                    username=form.username.data,
                    password=form.password.data)

        db.session.add(user)
        db.session.commit()
        flash('Thanks for registering! Now you can login!')
        return redirect(url_for('login'))
    return render_template('register.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)



class Auth:
    def __init__(self, user, pwd, fer_key):
        self.user = user
        self.pwd = pwd
        self.fer_key = fer_key
    
    
    def credentials(self):
        with open("User_Details.txt", "rb") as file:
            data = file.readlines()
            file.close()
            
            x = False
            y = 0
            while x == False and y < len(data):
                if (self.fer_key.decrypt(self.user) == self.fer_key.decrypt(data[y])) and (self.fer_key.decrypt(self.pwd) == self.fer_key.decrypt(data[y+1])):
                    x = True
                    return True
                else:
                    y += 2
            
            if x == False:
                return False
            
        
    def new_user(self):
        file = open("User_Details.txt", "ab")

        file.write(self.user)
        line = str(0) + "\n"
        file.write(line.encode('utf-8'))

        file.write(self.pwd)
        line = str(0) + "\n"
        file.write(line.encode('utf-8'))

        file.close()

        return 'User created successfully!'



class Keys:
    def key_gen():
        key = Fernet.generate_key()

        file = open("Fernet_Key.txt", "wb")
        file.write(key)
        file.close()

        print('New key generated successfully!')


    def read_key():
        with open('Fernet_Key.txt', 'rb') as file:
            data = file.read()
            file.close()

        return data



class LoadData:

    def __init__(self, agent_name, vc_number, receipt_number=None, package_name=None, amount_received=None, remarks_any=None):
        self.agent_name = agent_name
        self.vc_number = vc_number
        self.receipt_number = receipt_number
        self.package_name = package_name
        self.amount_received = amount_received
        self.remarks_any = remarks_any

        self.dir_path = "/Users/srikanthshileshpasam/Documents/GitHub/PSRK_CollectionAPP/Data/"
        self.master_file_name = 'TVR_Siti_Collection_Form_Master.csv'
        self.monthly_file_name = 'TVR_Siti_Collection_Form_Monthly.csv'


    def vc_check(self):
        vc_read = pd.read_csv(self.dir_path + self.master_file_name)

        data_df = pd.DataFrame(vc_read, columns = ['VC No', 'LCO ID', 'Name', 'Due Date', 'Cel 1'])

        vc_result = data_df["VC No"].isin([self.vc_number]) 
        data_df = data_df[vc_result]

        if data_df.empty == True:
            return False

        else:
            return data_df


    def file_open(self):    
        monthly_data = pd.read_csv(self.dir_path + self.monthly_file_name)

        class_call = DataEntry(monthly_data, self.dir_path, self.monthly_file_name)
        send_details = class_call.write_data(self.agent_name, self.vc_number, self.receipt_number, self.package_name, self.amount_received, self.remarks_any)

        return send_details




class DataEntry:
    def __init__(self, monthly, dir_name, monthly_name):
        self.monthly = monthly
        self.dir_name = dir_name
        self.monthly_name = monthly_name


    def write_data(self, agent_id, vc, receipt, package, amount, rem):
        self.agent_id = agent_id
        self.vc = vc
        self.receipt = receipt
        self.package = package
        self.amount = amount
        self.rem = rem

        date_time = datetime.now().strftime('%d-%m-%y %H:%M')
        auto_bill_num = random.randint(10000,99999999) 
#         need to develop


        self.monthly = self.monthly.append({'VC No':vc, 'Rec No':receipt, 'PKG':package, 'Recd Amt':amount, 'Coll Point':agent_id, 'Remarks':rem, 'Auto Bill No':auto_bill_num, 'Date':date_time}, ignore_index=True)

        print('\nSaving file..\n')

        self.monthly.to_csv(self.dir_name + self.monthly_name, index=False)

        return True




# Keys.key_gen()


login = int(input('\nWelcome!\nAre you an existing user or a new user?\n1) Existing user\n2) New user\n'))

if login == 1:
    user_id = input('\nUsername: ')
    user_id_enc = user_id.encode()
    pwd = getpass('Password: ')
    pwd = pwd.encode()

    key_code = Keys
    code = key_code.read_key()

    key = Fernet(code)

    encrypted_pwd = key.encrypt(pwd)
    encrypted_id = key.encrypt(user_id_enc)

    user_auth = Auth(encrypted_id, encrypted_pwd, key)

    login_status = user_auth.credentials()


    if login_status == True:

        vc_num = input('\nEnter VC number:\n')
        while vc_num[0] != '0' and len(vc_num) > 10 and len(vc_num) < 5:
            vc_num = input('\nINVALID VC!\nTry again..\nEnter VC number:\n')

        start_program = LoadData(user_id, vc_num)
        vc_exist = start_program.vc_check()

        if isinstance(vc_exist, pd.DataFrame):
            print(f'\nVC details found.\nVerify customer details before proceeding.\n\n{vc_exist}')

            receipt_num = int(input('\nEnter Receipt number:\n'))
            while receipt_num > 99999:
                receipt_num = int(input('\nINVALID RECEIPT!\nTry again..\nEnter Receipt number:\n'))

            package_type = int(input('\nSelect Package type:\n1) PKG 300\n2) PKG 250\n3) PKG 230\n4) PKG 200\n5) Others..\n'))
            while package_type != 1 and package_type != 2 and package_type != 3 and package_type != 4 and package_type !=5:
                package_type = int(input('\nINVALID PACKAGE!\nTry again..\nSelect Package type:\n1) PKG 300\n2) PKG 250\n3) PKG 230\n4) PKG 200\n5) Others..'))

            if package_type == 5:
                package_type = input('\nEnter Package details\n')

            elif package_type == 1:
                package_type = 'PKG 300'

            elif package_type == 2:
                package_type = 'PKG 250'

            elif package_type == 3:
                package_type = 'PKG 230'

            elif package_type == 4:
                package_type = 'PKG 200'


            amt_paid = int(input('\nEnter the amount paid by the customer:\n'))
            while amt_paid > 9999:
                amt_paid = int(input('\nINVALID AMOUNT!\nTry again..\nEnter the amount paid by the customer:\n'))

            remarks = False
            remarks_choice = int(input('\nAny remarks?\n1) Yes\n2) No\n'))
            if remarks_choice == 1:
                remarks = input('\nEnter remarks below:\n')

            else:
                print('\nData successfully captured!')


            start_program = LoadData(user_id, vc_num, receipt_num, package_type, amt_paid, remarks)                         
            writing_db = start_program.file_open()

            if writing_db == True:
                print('\nSave Successful!')

            else:
                print('\nERROR! FIle not saved!')

        else:
            print('\nVC data does not exist!\n')

    else:
        print('\nLogin failed!')


elif login == 2:
    user_id = input('\nCreate a user name: ')
    user_id = user_id.encode()
    pwd = getpass('Create your password: ')
    pwd = pwd.encode()

    key_code = Keys
    code = key_code.read_key()

    key = Fernet(code)

    encrypted_pwd = key.encrypt(pwd)
    encrypted_id = key.encrypt(user_id)

    user_auth = Auth(encrypted_id, encrypted_pwd)

    login_status = user_auth.new_user()

    print(f'\n{login_status}')

else:
    print('Invalid choice!')                             

In [3]:
import sqlite3
import pandas as pd
#Name of Excel xlsx file. SQLite database will have the same name and extension .db
filename="/Users/srikanthshileshpasam/Documents/GitHub/PSRK_CollectionAPP/Data/TVR_Siti_Collection_Form" 
con=sqlite3.connect(filename+".db")
wb=pd.read_excel(filename+'.xlsx')
for sheet in wb:
    wb[sheet].to_sql(sheet,con, index=False)
con.commit()
con.close()

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'Sms ID,V'